## DeepStream: Ejemplo para leer de archivo y escribir en archivo

In [ ]:
import os, sys

import gi
gi.require_version('Gst', '1.0')
from gi.repository import GLib, Gst

import pyds

sys.path.append('../')
sys.path.append('../apps/')
from common.bus_call import bus_call

### Archivos y configuración del primary nvinfer, copiada de test1

In [ ]:
input_file = "/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/apps/theGoat/datasheet/peatonesJapon.h264"
output_file = "output_simple.mp4"
config_file = "./configs/dstest1_pgie_config.txt"

import cv2
vid = cv2.VideoCapture(input_file)
input_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
input_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))

### Declaración del probe para dibujar la interfaz, copiado de deepstream_test_1.ipynb

In [ ]:
PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3

def osd_sink_pad_buffer_probe(pad,info,u_data):
    frame_number=0
    #Intiallizing object counter with 0.
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE:0,
        PGIE_CLASS_ID_PERSON:0,
        PGIE_CLASS_ID_BICYCLE:0,
        PGIE_CLASS_ID_ROADSIGN:0
    }
    num_rects=0

    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            # The casting is done by pyds.glist_get_nvds_frame_meta()
            # The casting also keeps ownership of the underlying memory
            # in the C code, so the Python garbage collector will leave
            # it alone.
            frame_meta = pyds.glist_get_nvds_frame_meta(l_frame.data)
        except StopIteration:
            break

        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        while l_obj is not None:
            try:
                # Casting l_obj.data to pyds.NvDsObjectMeta
                obj_meta=pyds.glist_get_nvds_object_meta(l_obj.data)
            except StopIteration:
                break
            obj_counter[obj_meta.class_id] += 1
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break

        # Acquiring a display meta object. The memory ownership remains in
        # the C code so downstream plugins can still access it. Otherwise
        # the garbage collector will claim it when this probe function exits.
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Setting display text to be shown on screen
        # Note that the pyds module allocates a buffer for the string, and the
        # memory will not be claimed by the garbage collector.
        # Reading the display_text field here will return the C address of the
        # allocated string. Use pyds.get_string() to get the string content.
        py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Vehicle_count={} Person_count={}".format(frame_number, num_rects, obj_counter[PGIE_CLASS_ID_VEHICLE], obj_counter[PGIE_CLASS_ID_PERSON])

        # Now set the offsets where the string should appear
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12

        # Font , font-color and font-size
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 10
        # set(red, green, blue, alpha); set to White
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)

        # Text background color
        py_nvosd_text_params.set_bg_clr = 1
        # set(red, green, blue, alpha); set to Black
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        # Using pyds.get_string() to get display_text as string
        print(pyds.get_string(py_nvosd_text_params.display_text))
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
			
    return Gst.PadProbeReturn.OK

### Declaración de los elementos lo más simple posible, sin tracker ni secondaries ni na

In [ ]:
Gst.init(None)

print("Creating elements...")
pipeline = Gst.Pipeline()

source = Gst.ElementFactory.make("filesrc", "file-source")
source.set_property('location', input_file)
pipeline.add(source)

parser = Gst.ElementFactory.make("h264parse", "h264-parser")
pipeline.add(parser)

decoder = Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")
pipeline.add(decoder)

streammux = Gst.ElementFactory.make("nvstreammux", "stream-muxer")
streammux.set_property('width', input_width)
streammux.set_property('height', input_height)
streammux.set_property('batched-push-timeout', 4000000)
streammux.set_property('batch-size', 1)
pipeline.add(streammux)

pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
pgie.set_property('config-file-path', config_file)
pipeline.add(pgie)

nvvidconv = Gst.ElementFactory.make("nvvideoconvert", "convertor")
pipeline.add(nvvidconv)

nvosd = Gst.ElementFactory.make("nvdsosd", "onscreendisplay")
osdsinkpad = nvosd.get_static_pad("sink")
osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)
pipeline.add(nvosd)

nvvidconv2 = Gst.ElementFactory.make("nvvideoconvert", "convertor2")
pipeline.add(nvvidconv2)

encoder = Gst.ElementFactory.make("x264enc", "encoder")
encoder.set_property('bitrate', 4000) # Baja esto para vídeos más pequeños pero de peor calidad
pipeline.add(encoder)

parser2 = Gst.ElementFactory.make("h264parse", "h264-parser2")
pipeline.add(parser2)

mux = Gst.ElementFactory.make("qtmux", "muxer")
pipeline.add(mux)

# Esto por si quieres que salga por pantalla.
#print("Creating eglsink...")
#eglsink = Gst.ElementFactory.make("nveglglessink", "nvvideo-renderer")
#pipeline.add(eglsink)

filesink = Gst.ElementFactory.make("filesink", "file-sink")
filesink.set_property('location', output_file)
filesink.set_property("sync", 0) # Don't sync to clock, to allow samples to be played as fast as possible
pipeline.add(filesink)

print("Linking elements...")
source.link(parser)
parser.link(decoder)

# decoder -> streammux
sinkpad = streammux.get_request_pad("sink_0")
srcpad = decoder.get_static_pad("src")
srcpad.link(sinkpad)

streammux.link(pgie)
pgie.link(nvvidconv)
nvvidconv.link(nvosd)
#nvosd.link(eglsink) # Si quieres que salga por pantalla descomenta este y comenta todos los que hay debajo.
nvosd.link(nvvidconv2)
nvvidconv2.link(encoder)
encoder.link(parser2)
parser2.link(mux)
mux.link(filesink)

### Ejecutamos el pipeline

In [ ]:
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop)

print("Running pipeline...")
pipeline.set_state(Gst.State.PLAYING)
loop.run()

pipeline.set_state(Gst.State.NULL)

In [ ]:
from IPython.display import Video
Video(output_file)